In [1]:
import pandas as pd

In [10]:
df = pd.read_csv("../data/dados_ench/API/api_ench_com_features.csv", index_col="date")

# Filtrar as linhas onde a coluna 'Nível' não é nulo
df = df[df["Nível"].notna()]

In [11]:
df.head()

,temperature_2m,relative_humidity_2m,rain,cloud_cover,wind_speed_10m,Nível,temperature_2m_media_movel_4h,temperature_2m_lag_4,temperature_2m_media_movel_8h,temperature_2m_lag_8,...,cloud_cover_media_movel_24h,cloud_cover_lag_24,wind_speed_10m_media_movel_4h,wind_speed_10m_lag_4,wind_speed_10m_media_movel_8h,wind_speed_10m_lag_8,wind_speed_10m_media_movel_12h,wind_speed_10m_lag_12,wind_speed_10m_media_movel_24h,wind_speed_10m_lag_24
date,,,,,,,,,,,,,,,,,,,,,
2023-09-29 03:00:00,14.707,95.57505,0.0,100.0,8.350138,4.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-29 04:00:00,14.657,94.95529,0.0,100.0,7.172949,4.73,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-29 05:00:00,14.757,92.82497,0.0,99.0,7.754637,4.70,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-29 06:00:00,14.707,92.82228,0.0,100.0,9.000000,4.67,14.7070,NaN,NaN,NaN,...,NaN,NaN,8.069431,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-29 07:00:00,14.457,93.72021,0.0,100.0,7.568566,4.64,14.6445,14.707,NaN,NaN,...,NaN,NaN,7.874038,8.350138,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df["Nível"]

date
2023-09-29 03:00:00    4.75
2023-09-29 04:00:00    4.73
2023-09-29 05:00:00    4.70
2023-09-29 06:00:00    4.67
2023-09-29 07:00:00    4.64
                       ... 
2023-10-06 22:00:00    6.77
2023-10-06 23:00:00    6.74
2023-10-07 00:00:00    6.71
2023-10-07 01:00:00    6.68
2023-10-07 02:00:00    6.67
Name: Nível, Length: 192, dtype: float64

In [14]:
df["Nível"].shift(-3)

date
2023-09-29 03:00:00    4.67
2023-09-29 04:00:00    4.64
2023-09-29 05:00:00    4.61
2023-09-29 06:00:00    4.58
2023-09-29 07:00:00    4.54
                       ... 
2023-10-06 22:00:00    6.68
2023-10-06 23:00:00    6.67
2023-10-07 00:00:00     NaN
2023-10-07 01:00:00     NaN
2023-10-07 02:00:00     NaN
Name: Nível, Length: 192, dtype: float64

In [15]:
df["target"] = df["Nível"].shift(-3)

In [16]:
df.head()

,temperature_2m,relative_humidity_2m,rain,cloud_cover,wind_speed_10m,Nível,temperature_2m_media_movel_4h,temperature_2m_lag_4,temperature_2m_media_movel_8h,temperature_2m_lag_8,...,cloud_cover_lag_24,wind_speed_10m_media_movel_4h,wind_speed_10m_lag_4,wind_speed_10m_media_movel_8h,wind_speed_10m_lag_8,wind_speed_10m_media_movel_12h,wind_speed_10m_lag_12,wind_speed_10m_media_movel_24h,wind_speed_10m_lag_24,target
date,,,,,,,,,,,,,,,,,,,,,
2023-09-29 03:00:00,14.707,95.57505,0.0,100.0,8.350138,4.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.67
2023-09-29 04:00:00,14.657,94.95529,0.0,100.0,7.172949,4.73,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.64
2023-09-29 05:00:00,14.757,92.82497,0.0,99.0,7.754637,4.70,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.61
2023-09-29 06:00:00,14.707,92.82228,0.0,100.0,9.000000,4.67,14.7070,NaN,NaN,NaN,...,NaN,8.069431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.58
2023-09-29 07:00:00,14.457,93.72021,0.0,100.0,7.568566,4.64,14.6445,14.707,NaN,NaN,...,NaN,7.874038,8.350138,NaN,NaN,NaN,NaN,NaN,NaN,4.54
